In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
  

"""
fe : ratio of ethanol 
fpg : ratio of PG
Cm : experimental data of solubiluty in mixter based on ratio
Cw : solubility in pure water
Cpg : solubility in pure PG
Ce : solubility in pure ethanol
T : temperature
LCm,LCe, ... : log of Cm, ...

x , y : for finding j with regression

Sm : prediction data of solubility 
LSm : prediction data of log Sm

"""

# Binary mixture

In [ ]:

# function for change molar to mole fraction
# density of solvent
de = 0.785
dw = 0.997
dpg = 1.0273
    # solvent Molecular weight
mw_w = 18
mw_e = 46
mw_pg = 76.09
def Mole_fraction (M, d1, d2, mw_1, mw_2 ):
    if len(M) == 7 :
        f1 = [1,0.8,0.6,0.5,0.4,0.2,0]
        f2 = [0,0.2,0.4,0.5,0.6,0.8,1]
        f1 = np.array(f1).reshape(-1,1)
        f2= np .array(f2).reshape(-1,1)
    else:
        f1 = np.arange(0,1.01, 0.01)
        f2 = 1-f1
    
    mol_1 = (f1*d1)/mw_1
    mol_2 = (f2*d2)/mw_2
    X2 = mol_2/(mol_2+mol_1)
    X1 = mol_1/(mol_2+mol_1)
    MW = X2*mw_2 + X1*mw_1
    d = (f1*d1) + (f2*d2)
    m = (M/(1000*d))*MW
    Xe = m/(m+1)
    return (Xe, MW, d)
    #print('exp mole fraction = ' , Xe)
   

In [ ]:
#   jouyban-Acree function for binary mixtures

def Jouyban_Acree (Cm, C1, C2, d1, d2):
    F1 = [1,0.8,0.6,0.5,0.4,0.2,0]
    F2 = [0,0.2,0.4,0.5,0.6,0.8,1]
    F1 = np.array(F1).reshape(-1,1)
    F2= np .array(F2).reshape(-1,1)
    # change v/v to w/w
    W1 = F1*d1
    W2 = F2*d2
    f1 = W1/(W2+W1)
    f2 = W2/(W2+W1)
    
    LCm = [math.log10(x) for x in Cm]
    LC1 = math.log10(C1)
    LC2 = math.log10(C2)
    LCm = np.array(LCm).reshape(-1,1)

    x = LCm - (f1 * LC1 + f2 * LC2) 
    y0 = (f1 * f2)/T
    y1 = (f1*f2*(f1-f2))/T
    y2 = (f1*f2*(f1-f2)*(f1-f2))/T

    reg0 = LinearRegression().fit(y0,x)
    reg1 = LinearRegression().fit(y1,x)
    reg2 = LinearRegression().fit(y2,x)

    j0 = float(reg0.coef_)
    j1 = float(reg1.coef_)
    j2 = float(reg2.coef_)
    #print('j0 = ',j0 , '\n' ,'j1 = ',j1 , '\n' , 'j2 = ',j2)
    f12 = np.arange(0,1.01, 0.01)
    f22 = 1-f12

    LSm = (f12 * LC1) + (f22 * LC2) + (f12*f22/T)*(j0 + (j1 * (f12-f22)) + (j2*(f12-f22)*(f12-f22)))
    Sm = 10**LSm
    #print (Sm)
    return (Sm)




# Ternary mixture

In [ ]:
# function for change molar to mole fraction
# density of solvent
de = 0.785
dw = 0.997
dpg = 1.0273
    # solvent Molecular weight
mw_w = 18
mw_e = 46
mw_pg = 76.09
def Mole_fraction (M, d1, d2, d3, mw_1, mw_2, mw_3 ):
    if len(M) <= 10 :
        f1 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
        f2 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
        f3 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
        f1 = np.array(f1).reshape(-1,1)
        f2= np .array(f2).reshape(-1,1)
        f3= np .array(f3).reshape(-1,1)

    else:
        f1 = np.arange(0.1,0.8, 0.01)
        f2 = (1-f1)/2
        f3 = (1-f1)/2
    
    mol_1 = (f1*d1)/mw_1
    mol_2 = (f2*d2)/mw_2
    mol_3 = (f3*d3)/mw_3    
    X1 = mol_1/(mol_2+mol_1+mol_3)
    X2 = mol_2/(mol_2+mol_1+mol_3)
    X3 = mol_3/(mol_2+mol_1+mol_3)
    MW = X1*mw_1 + X2*mw_2 + X3*mw_3
    d = (f1*d1) + (f2*d2) + (f3*d3)
    m = (M/(1000*d))*MW
    Xe = m/(m+1)
    return (Xe, MW, d)
    #print('exp mole fraction = ' , Xe)


#   jouyban-Acree function for ternary mixture

def Jouyban_Acree (Cm, C1, C2, C3, d1, d2, d3):
    F1 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
    F2 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
    F3 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
    F1 = np.array(F1).reshape(-1,1)
    F2= np .array(F2).reshape(-1,1)
    F3= np .array(F3).reshape(-1,1)
    
    # change v/v to w/w
    W1 = F1*d1
    W2 = F2*d2
    W3 = F3*d3
    f1 = W1/(W2+W1+W3)
    f2 = W2/(W2+W1+W3)
    f3 = W3/(W2+W1+W3)
    
    LCm = [math.log10(x) for x in Cm]
    LCm = np.array(LCm).reshape(-1,1)
    LC1 = math.log10(C1)
    LC2 = math.log10(C2)
    LC3 = math.log10(C3)
    

    x = LCm - ((f1 * LC1) + (f2 * LC2) + (f3 * LC3))
    y10 = (f1 * f2)/T
    y11 = (f1*f2*(f1-f2))/T
    y12 = (f1*f2*(f1-f2)*(f1-f2))/T
    reg10 = LinearRegression().fit(y10,x)
    reg11 = LinearRegression().fit(y11,x)
    reg12 = LinearRegression().fit(y12,x)
    j10 = float(reg10.coef_)
    j11 = float(reg11.coef_)
    j12 = float(reg12.coef_)
    #print('j0 = ',j0 , '\n' ,'j1 = ',j1 , '\n' , 'j2 = ',j2)
    
    y20 = (f1 * f3)/T
    y21 = (f1*f3*(f1-f3))/T
    y22 = (f1*f3*(f1-f3)*(f1-f3))/T
    reg20 = LinearRegression().fit(y20,x)
    reg21 = LinearRegression().fit(y21,x)
    reg22 = LinearRegression().fit(y22,x)
    j20 = float(reg20.coef_)
    j21 = float(reg21.coef_)
    j22 = float(reg22.coef_)
    #print('j20 = ',j20 , '\n' ,'j21 = ',j21 , '\n' , 'j22 = ',j22)

    y30 = (f2 * f3)/T
    y31 = (f2*f3*(f2-f3))/T
    y32 = (f2*f3*(f2-f3)*(f2-f3))/T
    reg30 = LinearRegression().fit(y30,x)
    reg31 = LinearRegression().fit(y31,x)
    reg32 = LinearRegression().fit(y32,x)
    j30 = float(reg30.coef_)
    j31 = float(reg31.coef_)
    j32 = float(reg32.coef_)
    #print('j30 = ',j30 , '\n' ,'j31 = ',j31 , '\n' , 'j32 = ',j32)
    '''
    y40 = (f1 * f2 * f3)/T
    y41 = (f1*f2*f3*(f1-f2-f3))/T
    y42 = (f1*f2*f3*(f1-f2-f3)*(f1-f2-f3))/T
    reg40 = LinearRegression().fit(y40,x)
    reg41 = LinearRegression().fit(y41,x)
    reg42 = LinearRegression().fit(y42,x)
    j40 = float(reg40.coef_)
    j41 = float(reg41.coef_)
    j42 = float(reg42.coef_)
    print('j40 = ',j40 , '\n' ,'j41 = ',j41 , '\n' , 'j42 = ',j42)
    '''
    
    F12 = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
    F22 = [0.1,0.2,0.2,0.3,0.3,0.4,0.4,0.5]
    F32 = [0.1,0.1,0.2,0.2,0.3,0.3,0.4,0.4]
    F12 = np.array(F12).reshape(-1,1)
    F22= np .array(F22).reshape(-1,1)
    F32= np .array(F32).reshape(-1,1)
        
    # change v/v to w/w
    W1 = F12*d1
    W2 = F22*d2
    W3 = F32*d3
    f12 = W1/(W2+W1+W3)
    f22 = W2/(W2+W1+W3)
    f32 = W3/(W2+W1+W3)    
    
    f12 = np.arange(0.1,0.8, 0.01)
    f22 = (1-f12)/2
    f32 = (1-f12)/2


    A1 = (f12*f22/T)*(j10 + (j11 * (f12-f22)) + (j12*(f12-f22)*(f12-f22)))
    A2 = (f12*f32/T)*(j20 + (j21 * (f12-f32)) + (j22*(f12-f32)*(f12-f32)))
    A3 = (f22*f32/T)*(j30 + (j31 * (f22-f32)) + (j32*(f22-f32)*(f22-f32)))
    #A4 = (f12*f22*f32/T)*(j40 + (j41 * (f12-f22-f32)) + (j42*(f12-f22-f32)*(f12-f22-f32)))


    LSm = ((f12 * LC1) + (f22 * LC2) + (f32 * LC3)) + A1 + A2 + A3 
    Sm = 10**LSm

    #print ('X_calc =', Sm)
    return (Sm)
